In [2]:
from utils.tf import *
from utils.db import *
from utils.gd import * 
from utils.clt_repetidos import *
from datetime import datetime
import cx_Oracle
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from utils.clt_repetidos import * 
from ydata_profiling import ProfileReport
import dask.dataframe as dd
import gc
import matplotlib.pyplot as plt

In [3]:
path = "/home/ale1726/proyects/datalake/clientes/data/productos/FFON"

In [3]:
agrupado_sistemas_transacciones_general = pd.read_csv(os.path.join(path,"agrupado_sistemas_transacciones_general.dat"))

In [6]:
agrupado_sistemas_transacciones_general

,CFE_C_SISTEMA,NOMBRE_SISTEMA,SISTEMA,NUM_TRANSACCIONES,MEDIA_MONTO,SUMA_MONTO,MEDIANA_MONTO
0,2050,SISTEMA SIRAC,SIRAC,164565,1.162676e+07,1.913358e+12,1485.75
1,2051,NAFIN ELECTRONICO,NAFIN,188706,0.000000e+00,0.000000e+00,0.00
2,2052,SISTEMA TAS,TAS,1533975,6.861308e+08,1.052508e+15,2985478.33
3,2053,SISTEMA CUSTODIA DE VALORES,CUSTODIA,1086272,6.845386e+07,7.435951e+13,0.00
4,2054,SISTEMA IKOS CASH,FFON,4114656,4.493387e+07,1.848874e+14,40.00
5,2055,SISTEMA IKOS CASH WEB,WEB FFON,111479,1.069208e+07,1.191942e+12,6144.99
6,39005,OPERACIONES EN FORMA MASIVA VIA WEB,CARGA MASIVA,6,0.000000e+00,0.000000e+00,0.00
7,109691,SISTEMA DE MERCADO DE CAPITALES,MECA,13949,8.121841e+05,1.132916e+10,3.22
8,128716,SISTEMA DE VENTAS EN DIRECTO,SVD,54780080,6.700585e+03,3.670586e+11,200.00
9,174039,INVERSIONES Y PRESTAMOS EXTRANJEROS,SIPE,31401,1.184569e+07,3.719664e+11,0.00


In [8]:
agrupado_sistemas_transacciones_general["MEDIA_MONTO"].mean()

63554766.11423721

In [ ]:
63,554,766

In [ ]:
agrupado_sistemas_transacciones_general.sort_values(by="")

In [4]:
agrupado_conceptos_entrada = pd.read_csv(os.path.join(path,"agrupado_conceptos_entrada.dat"))

In [ ]:
agrupado_conceptos_entrada.

,NUM_CONCEPTO,CONCEPTO,NUM_OPERACIONES,MEDIA_MONTO,SUMA_MONTO
0,128723,INGRESO DOMI SVD,26099582,1.385172e+03,3.615240e+10
1,147387,EGRESOS SPEI SVD,16266567,0.000000e+00,0.000000e+00
2,147384,INGRESO SPEI SVD,12247829,2.688088e+04,3.292324e+11
3,8469,TRASPASO ENTRADA,2069460,8.923770e+07,1.846738e+14
4,8305,TRASPASOS_SAL,2021477,0.000000e+00,0.000000e+00
...,...,...,...,...,...
220,6068,RECEPCION DE COMPENSACION,1,1.956325e+06,1.956325e+06
221,36695,PAGO A PROVEEDORES NORMATIVIDAD GUBERNAMENTAL,1,0.000000e+00,0.000000e+00
222,37982,CANCELACION PAGO A PROVEEDOR CON CHEQUE NORMAT...,1,1.250000e+03,1.250000e+03
223,37809,PAGO CON CHEQUE A PROV EMPRENDEDORES,1,0.000000e+00,0.000000e+00


In [13]:
agrupado_conceptos_entrada
top5 = agrupado_conceptos_entrada.nlargest(5, "NUM_OPERACIONES")
total = agrupado_conceptos_entrada["NUM_OPERACIONES"].sum()
res = [[row["CONCEPTO"], round(row["NUM_OPERACIONES"] / total * 100)] for _, row in top5.iterrows()]
otros = round(100 - sum(p[1] for p in res))
res.append(["otros", otros])


In [14]:
res

[['INGRESO DOMI SVD', 42],
 ['EGRESOS SPEI SVD', 26],
 ['INGRESO SPEI SVD', 20],
 ['TRASPASO ENTRADA', 3],
 ['TRASPASOS_SAL', 3],
 ['otros', 6]]

In [6]:
# Obtener los 5 conceptos con más transacciones
top5 = agrupado_conceptos_entrada.sort_values(by="NUM_OPERACIONES", ascending=False).head(5)

# Calcular el total de operaciones en todo el DataFrame
total_operaciones = agrupado_conceptos_entrada["NUM_OPERACIONES"].sum()

# Calcular el total de operaciones de los 5 conceptos principales
top5_total = top5["NUM_OPERACIONES"].sum()

# Calcular el porcentaje que representan
porcentaje = (top5_total / total_operaciones) * 100

print("Top 5 conceptos con más transacciones:")
print(top5)

print(f"\nEstas transacciones representan el {porcentaje:.2f}% del total.")


Top 5 conceptos con más transacciones:
   NUM_CONCEPTO          CONCEPTO  NUM_OPERACIONES   MEDIA_MONTO    SUMA_MONTO
0        128723  INGRESO DOMI SVD         26099582  1.385172e+03  3.615240e+10
1        147387  EGRESOS SPEI SVD         16266567  0.000000e+00  0.000000e+00
2        147384  INGRESO SPEI SVD         12247829  2.688088e+04  3.292324e+11
3          8469  TRASPASO ENTRADA          2069460  8.923770e+07  1.846738e+14
4          8305     TRASPASOS_SAL          2021477  0.000000e+00  0.000000e+00

Estas transacciones representan el 94.21% del total.
